### 项目描述
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

Rossmann公司在7个欧洲国家一共拥有3000家药妆店。目前Rossmann商店经理需要预测未来6周的销售额。商店的销售额会受到许多因素的影响，包括促销，竞争对手，学校，节假日，季度和区域性。由上千位背景不同的经理基于他们的环境而预测出的销售额差别是非常大的。

In their first Kaggle competition, Rossmann is challenging you to predict 6 weeks of daily sales for 1,115 stores located across Germany. Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust prediction model, you will help store managers stay focused on what’s most important to them: their customers and their teams! 

在Rossmann的第一次Kaggle竞赛中，它需要你为它预测德国境内的1115家药妆店6周内每天的销售额。可靠的销售额可以激发员工的工作积极性和产出。通过帮助Rossmann创建一个可靠，强壮的预测模型，你可以帮助Rossmann的经理们关注对他们来说最重要的东西：客户和团队。


### 问题描述
  项目选自Kaggle的一个竞赛项目 [Rossmann Store Sales](https://www.kaggle.com/c/rossmann-store-sales)，目标是预测Rossmann的销售额。是一个监督学习的回归类的问题，可以用LinearRegression，XGBoost，神经网络等机器学习的算法来预测销售额。
  
### 数据集的下载
数据全部可以从[Kaggle](https://www.kaggle.com/c/rossmann-store-sales/data)上下载。一共提供了4个文件：

- train.csv - historical data including Sales (训练集，包括了销售额)
- test.csv - historical data excluding Sales （测试集，没有包括销售额）
- sample_submission.csv - a sample submission file in the correct format （最终需要提交的文件模板）
- store.csv - supplemental information about the stores （补充信息）

#### 数据描述

- Id - an Id that represents a (Store, Date) duple within the test set（测试集中的唯一索引，由商店和日期表示，可以理解为数据库中的联合主键）
- Store - a unique Id for each store （商店的唯一索引）
- Sales - the turnover for any given day (this is what you are predicting) （销售额，需要在测试集中预测）
- Customers - the number of customers on a given day （当天的客户数量）
- Open - an indicator for whether the store was open: 0 = closed, 1 = open （是否营业，0.否 1.是）
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None （法定假日，大部分商店都会放假，但也有些列外，a = 公共假日，b = 复活节，c = 圣诞节， 0 = 无）
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools （学校假日，代表商店是否收到学校临时停课的影响）
- StoreType - differentiates between 4 different store models: a, b, c, d （商店类型）
- Assortment - describes an assortment level: a = basic, b = extra, c = extended （商店分类级别，a = 基本，b = 额外， c = 扩展）
- CompetitionDistance - distance in meters to the nearest competitor store （最近的竞争者的距离，单位：米）
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened （最近的竞争者，开始营业的月及年份）
- Promo - indicates whether a store is running a promo on that day （当天是否有促销）
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating（持续促销， 0 = 未参与， 1 = 参与）
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2 （持续促销开始的年/周数）
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store （促销的间隔）

至此我对项目有了一个大概的了解，接下来我们需要对数据进行探索。

### 数据的探索

- 首先利用pandas的API读取train,store,test3个文件，然后调用df的head方法，快速浏览下数据；
- train存放的是1151个商店在不同时期的销售额，而store存放的是各个商店的一些特征，test自然存放的是测试集；
- 为了方便训练和预测，我们肯定需要将train和test与store特征进行合并；
- 首先根据Key Store合并2个特征文件，然后调用head方法，看一下特征数据。
- Store，商店的ID从1至1155，需要做归一化处理；
- DayOfWeek，星期几1至7，需要做归一化处理；
- Date，日期，这个属性很特殊，需要转成其他多个属性，比如年，月，日，一个年的第几周，一个月的第几周，一年的第几天等；
- Sales，销量，提取为标签；
- Customers，客户数，直接drop掉；
- Open，是否营业，不做任何处理；
- Promo，是否促销，不做任何处理；
- StateHoliday，是否法定假日，不做任何处理；
- SchoolHoliday，是否学校放假，不做任何处理；
- StoreType，商店类型，需要独热编码；
- Assortment，销售种类，需要独热编码；
- CompetitionDistance，竞争对手距离，需要处理空值；
- CompetitionOpenSinceYeah，竞争对手营业年份，需要处理空值，然后做归一化处理；
- Promo2，是否有持续促销，不做任何处理；
- PromoSinceWeek，第几周开始促销的，需要处理空值并做归一化处理；
- Promo2SinceYear，哪一年开始促销的，需要处理空值并做归一化处理；
- PromoInterval，促销的间隔（由英文的月份表示），需要处理空值，同时需要拆分成多个属性；

In [26]:
import pandas as pd
import math

train = pd.read_csv('data/train.csv')
store = pd.read_csv('data/store.csv')
test = pd.read_csv('data/test.csv')
train.head()

c:\users\tracy\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [27]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [28]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [2]:
# 处理store中可能为NaN的特征
store['CompetitionDistance'] = store['CompetitionDistance'].apply(lambda x: 0 if math.isnan(x) else x)
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].apply(lambda x: 0 if math.isnan(x) else x)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].apply(lambda x: 0 if math.isnan(x) else x)
store['Promo2SinceYear'] = store['Promo2SinceYear'].apply(lambda x: 0 if math.isnan(x) else x)
store['Promo2SinceYear'] = store['Promo2SinceYear'].apply(lambda x: 0 if math.isnan(x) else x)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [10]:
# 合并特征
df = pd.merge(train, store, on='Store')


# 将数据切分成特征和对应的标签
sales_raw = df['Sales']
customer_raw = df['Customers']
date_raw = df['Date']
features_raw = df.drop(['Sales', 'Customers'], axis = 1)

features_raw.head(5)

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [25]:
#date_raw['Year'] = df['Date'].apply(lambda x: x.split('-')[0])

#date_raw['one_trunc'] = date_raw['Date'][:2]

#date_raw.head(5)



#df['Year'] = df['Date'].apply(lambda x: x.split('-')[0])
#df['Month'] = df['Date'].apply(lambda x: x.split('-')[1])
#df['Day'] = df['Date'].apply(lambda x: x.split('-')[2])
#df['WeekOfYear'] = df['Date'].apply(lambda x: x.split('-')[2])
#df['WeekOfMonth'] = df['Date'].apply(lambda x: x.split('-')[2])
#df['DayOfYear'] = df['Date'].apply(lambda x: x.split('-')[2])

#df['PromoInterval'] = df['PromoInterval'].apply(lambda x: 0 if math.isnan(x) else x)

df.head(3000)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,31
1,1,4,2015-07-30,5020,546,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,30
2,1,3,2015-07-29,4782,523,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,29
3,1,2,2015-07-28,5011,560,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,28
4,1,1,2015-07-27,6102,612,1,1,0,1,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,27
5,1,7,2015-07-26,0,0,0,0,0,0,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,26
6,1,6,2015-07-25,4364,500,1,0,0,0,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,25
7,1,5,2015-07-24,3706,459,1,0,0,0,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,24
8,1,4,2015-07-23,3769,503,1,0,0,0,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,23
9,1,3,2015-07-22,3464,463,1,0,0,0,c,...,1270.0,9.0,2008.0,0,0.0,0.0,NaN,2015,07,22


In [5]:
year = '2015-07-31'
print(year.split('-'))

['2015', '07', '31']


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 初始化一个 scaler，并将它施加到特征上
scaler = MinMaxScaler()
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
features_raw[numerical] = scaler.fit_transform(data[numerical])

# 显示一个经过缩放的样例记录
display(features_raw.head(n = 1))